In [1]:
import sys,os
sys.path.insert(1,'/home/nugrid/omega_sygma/')
os.environ["SYGMADIR"] = "/home/nugrid/omega_sygma/"
import matplotlib
%matplotlib inline

from modules.SYGMA import start_SYGMA

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [ ]:
import sys,os
sys.path.insert(1,'/rpod3/lsiemens/omega_sygma/')
os.environ["SYGMADIR"] = "/rpod3/lsiemens/omega_sygma/"
import matplotlib
%matplotlib inline

from modules.SYGMA import start_SYGMA

In [2]:
start_SYGMA()

default
